In [1]:
import pandas as pd
import yfinance as yf
from yahoofinancials import YahooFinancials as yfs
import json

In [12]:
#!pip install yahoofinancials

In [2]:
def Sheet_Data(ticker,sheet_data):
    save = []
    
    df = pd.DataFrame.from_dict({(i,j): sheet_data[i][j] 
                            for i in sheet_data.keys() 
                            for j in sheet_data[i].keys()},
                        orient='index')

    for i in df.columns:
        f = len(df.columns) - i - 1
        df2 = pd.json_normalize(df[f],max_level = 1)

        #Gets  Date and renames 
        Date = df2.columns[0]
        Date = Date[0:11]
        df2.index = df2.index.rename(ticker)
        df2.columns = df2.columns.str.replace(str(Date),'')

        #Tranposes DataFrame and changes it to the 
        df2 = df2.T
        df2.rename(columns = {df2.columns[0]:'{}'.format(str(Date[0:4]))},inplace = True)
        #display(df2)
        save.append(df2)
    df3 = pd.concat(save,axis = 1)
    display(df3)
    filename = str(df.index[0][1]) + '_' + str(df.index[0][0])
    print(filename)
    #df3.to_csv('{}.csv'.format(filename))
    return df3
#filename = str(df.index[0][1]) + '_' + str(df.index[0][0])
#df3.t

In [3]:
ticker = 'AAPL'
yahoo_financials = yfs(ticker)

cash = yahoo_financials.get_financial_stmts('annual','cash')
income = yahoo_financials.get_financial_stmts('annual','income')
balance = yahoo_financials.get_financial_stmts('annual','balance')

In [4]:
cash_balance = Sheet_Data(ticker,cash)
income_balance = Sheet_Data(ticker,income)
balance_balance = Sheet_Data(ticker,balance)

AAPL,2017,2018,2019,2020
investments,-33542000000,30845000000,58093000000,5335000000
changeToLiabilities,8373000000,9172000000,-2548000000,-1981000000
totalCashflowsFromInvestingActivities,-46446000000,16066000000,45896000000,-4289000000
netBorrowings,29014000000,432000000,-7819000000,2499000000
totalCashFromFinancingActivities,-17974000000,-87876000000,-90976000000,-86820000000
changeToOperatingActivities,-8480000000,30016000000,-896000000,881000000
issuanceOfStock,555000000,669000000,781000000,880000000
netIncome,48351000000,59531000000,55256000000,57411000000
changeInCash,-195000000,5624000000,24311000000,-10435000000
repurchaseOfStock,-34774000000,-75265000000,-69714000000,-75992000000


AAPL_cashflowStatementHistory


AAPL,2017,2018,2019,2020
researchDevelopment,11581000000,14236000000,16217000000,18752000000
effectOfAccountingCharges,None,None,None,None
incomeBeforeTax,64089000000,72903000000,65737000000,67091000000
minorityInterest,None,None,None,None
netIncome,48351000000,59531000000,55256000000,57411000000
sellingGeneralAdministrative,15261000000,16705000000,18245000000,19916000000
grossProfit,88186000000,101839000000,98392000000,104956000000
ebit,61344000000,70898000000,63930000000,66288000000
operatingIncome,61344000000,70898000000,63930000000,66288000000
otherOperatingExpenses,None,None,None,None


AAPL_incomeStatementHistory


AAPL,2017,2018,2019,2020
totalLiab,241272000000,258578000000,248028000000,258549000000
totalStockholderEquity,134047000000,107147000000,90488000000,65339000000
otherCurrentLiab,38099000000,39293000000,43242000000,47867000000
totalAssets,375319000000,365725000000,338516000000,323888000000
commonStock,35867000000,40201000000,45174000000,50779000000
otherCurrentAssets,13936000000,12087000000,12352000000,11264000000
retainedEarnings,98330000000,70400000000,45898000000,14966000000
otherLiab,43251000000,48914000000,50503000000,46108000000
treasuryStock,-150000000,-3454000000,-584000000,-406000000
otherAssets,18177000000,22283000000,32978000000,33952000000


AAPL_balanceSheetHistory


# Anaylsis

In [151]:
FCF = cash_balance.loc[['totalCashFromOperatingActivities','capitalExpenditures','netIncome']] # how to select rows 
Revenue = income_balance.loc[['totalRevenue','netIncome']]
Growth_Rate = Revenue.pct_change().mean()
#FCF is CFO - CAPEX = FCF
#FCF = cash_balance.loc[['totalCashFromOperatingActivities','capitalExpenditures','netBorrowings']] # More 

df = pd.DataFrame()
df['FCF'] = FCF.loc['totalCashFromOperatingActivities'] + FCF.loc['capitalExpenditures']
df['Net Income'] = FCF.loc['netIncome']
#df['FCF/Net Income'] = df['FCF']/df['Net Income']
minimum = (df['FCF']/df['Net Income']).min()


In [83]:

average_factor = cash_balance.pct_change(axis = 1 ).mean(axis = 1 )
average_factor = (100 + average_factor)/100
df = cash_balance
df['2021'] = cash_balance[cash_balance.columns[3]] * average_factor
for i in range(0,4,1):
    num =  int(df.columns[4]) + i

    df['{}'.format(num)] = df['{}'.format(num - 1)] * average_factor

df

AAPL,2017,2018,2019,2020,2021,2022,2023,2024
investments,-33542000000,30845000000,58093000000,5335000000,5.320032e+09,5.305107e+09,5.290223e+09,5.275381e+09
changeToLiabilities,8373000000,9172000000,-2548000000,-1981000000,-1.976984e+09,-1.972977e+09,-1.968977e+09,-1.964986e+09
totalCashflowsFromInvestingActivities,-46446000000,16066000000,45896000000,-4289000000,-4.285394e+09,-4.281792e+09,-4.278192e+09,-4.274595e+09
netBorrowings,29014000000,432000000,-7819000000,2499000000,2.421820e+09,2.347024e+09,2.274538e+09,2.204290e+09
totalCashFromFinancingActivities,-17974000000,-87876000000,-90976000000,-86820000000,-8.730589e+10,-8.779450e+10,-8.828585e+10,-8.877994e+10
changeToOperatingActivities,-8480000000,30016000000,-896000000,881000000,8.713990e+08,8.619026e+08,8.525097e+08,8.432191e+08
issuanceOfStock,555000000,669000000,781000000,880000000,8.806343e+08,8.812691e+08,8.819044e+08,8.825401e+08
netIncome,48351000000,59531000000,55256000000,57411000000,5.742744e+10,5.744388e+10,5.746032e+10,5.747677e+10
changeInCash,-195000000,5624000000,24311000000,-10435000000,-1.001420e+10,-9.610373e+09,-9.222829e+09,-8.850912e+09
repurchaseOfStock,-34774000000,-75265000000,-69714000000,-75992000000,-7.612146e+10,-7.625115e+10,-7.638105e+10,-7.651118e+10


In [157]:
Revenue
Revenue = Revenue.drop(['netIncome'])

KeyError: "['netIncome'] not found in axis"

In [170]:
#Anaylist esitmates 
df = pd.read_html('https://finance.yahoo.com/quote/AAPL/analysis?p=AAPL')
df = df[1] 
df.index = df[df.columns[0]]
df1 = df.index 
df1 = df[df.columns[3:5]]
df1 = df1.drop(['Sales Growth (year/est)'])

for i in df1.columns:
    df1[i] = df1[i].str.replace('B','e9')
    df1[i] = df1[i].astype(float)

#df1.loc['Avg. Estimate']
df1.loc['Avg. Estimate'].tolist()

Revenue
strs = int(Revenue.columns[3])
Revenue['{}'.format(strs+1)] = df1.loc['Avg. Estimate'][0]
Revenue['{}'.format(strs+2)] = df1.loc['Avg. Estimate'][1]
Revenue

AAPL,2017,2018,2019,2020,2021,2022
totalRevenue,229234000000,265595000000,260174000000,274515000000,3.336500e+11,3.481500e+11


In [24]:
{(i,j): cash[i][j] 
for i in cash.keys()
for j in cash[i].keys()
        }

{('cashflowStatementHistory',
  'AAPL'): [{'2020-09-26': {'investments': 5335000000,
    'changeToLiabilities': -1981000000,
    'totalCashflowsFromInvestingActivities': -4289000000,
    'netBorrowings': 2499000000,
    'totalCashFromFinancingActivities': -86820000000,
    'changeToOperatingActivities': 881000000,
    'issuanceOfStock': 880000000,
    'netIncome': 57411000000,
    'changeInCash': -10435000000,
    'repurchaseOfStock': -75992000000,
    'totalCashFromOperatingActivities': 80674000000,
    'depreciation': 11056000000,
    'otherCashflowsFromInvestingActivities': -791000000,
    'dividendsPaid': -14081000000,
    'changeToInventory': -127000000,
    'changeToAccountReceivables': 6917000000,
    'otherCashflowsFromFinancingActivities': -126000000,
    'changeToNetincome': 6517000000,
    'capitalExpenditures': -7309000000}}, {'2019-09-28': {'investments': 58093000000,
    'changeToLiabilities': -2548000000,
    'totalCashflowsFromInvestingActivities': 45896000000,
    'net

# Testing for working all of the files work

df = pd.DataFrame.from_dict({(i,j): balance_sheet_data[i][j] 
                           for i in balance_sheet_data.keys() 
                           for j in balance_sheet_data[i].keys()},
                       orient='index')
for i in df.columns:
    df[i].tolist()
    df2 = pd.json_normalize(df[i],max_level = 1)

    #Gets  Date and renames 
    Date = df2.columns[0]
    Date = Date[0:11]
    df2.index = df2.index.rename(ticker)
    df2.columns = df2.columns.str.replace(str(Date),'')

    #Tranposes DataFrame and changes it to the 
    df2 = df2.T
    df2.rename(columns = {df2.columns[0]:'{}'.format(str(Date[0:10]))},inplace = True)
    #display(df2)
    save.append(df2)
df3 = pd.concat(save,axis = 1)
display(df3)